In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_classifier3
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams):
    file = open("results/gridsearch_selection.txt","a") 
    #file = open("results/gridsearch_selection_TEST.txt","a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading sinthetic dataset
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D]
    #real datasets
    #datasets = [setup.loadKeystroke] #, setup.loadCheckerBoard, setup.loadNOAADataset, setup.loadElecData
    initialLabeledData = 50
    #datasets = [ setup.loadKeystroke]
    batches=100
    poolSize = None
    isBatchMode = True
    #testing grid search
    for i in range(len(datasets)):
        finalScore = 0
        best_grid={}
        dataValues, dataLabels, description = datasets[i](path, sep)
        sizeOfBatch = int((len(dataValues)-initialLabeledData)/batches)
        print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))
        tuned_params = [{"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], "sizeOfBatch":[sizeOfBatch], "batches":[batches], "poolSize":[poolSize], "isBatchMode":[isBatchMode], "initialLabeledData":[initialLabeledData]}]
        #tuned_params = [{"excludingPercentage" : [0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5, 0.05], "sizeOfBatch":[sizeOfBatch], "batches":[batches]}]

        for g in ParameterGrid(tuned_params):
            averageAccuracy=0
            gs = grid_selection_classifier3.label_propagation(**g)
            try:
                gs.fit(dataValues, dataLabels)
                averageAccuracy = np.mean(gs.predict())
                if finalScore < averageAccuracy:
                    finalScore = averageAccuracy
                    best_grid = g
            except Exception:
                print("An error occured in ", description, g)
                raise Exception
            
        print(finalScore)
        print(best_grid)
        
        writeResults(description, finalScore, best_grid)
    
if __name__ == "__main__":
    main()


One Class Diagonal Translation. 2 Dimensional data.: 100 batches of 159 instances
99.8176100629
{'K': 5, 'batches': 100, 'initialLabeledData': 50, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 159}
One Class Horizontal Translation. 2 Dimensional data.: 100 batches of 159 instances
99.3522012579
{'K': 9, 'batches': 100, 'initialLabeledData': 50, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 159}
Two Classes Diagonal Translation. 2 Dimensional data: 100 batches of 159 instances
92.6226415094
{'K': 10, 'batches': 100, 'initialLabeledData': 50, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 159}
Two Classes Horizontal Translation. 2 Dimensional data.: 100 batches of 159 instances
54.0314465409
{'K': 2, 'batches': 100, 'initialLabeledData': 50, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 159}
Four Classes Rotating Separated. Bidimensional.: 100 batches of 1443 instances
99.9847539848
{'K': 8, 'batches': 100, 'initialLabeledData': 50, 'isBatchMode': True, 'po